# Introduction to the Evaluation Class

## Introduction
In this notebook we will continue to explore the Evaluation schema through the Evaluation class interface. To do so, we first need to create an Evaluation and populate it with data. There are many ways to do this ranging from cloneing a complete Evaluation from the TEEHR S3 bucket that already contains all the nessesary data, to cloning a blank template and populating the tables with all the nessesary data using the builtin loading and fetching methods.  In this exercise we are going to clone a complete Evaluation and explore the tables using the TEEHR Evaluation table subclasses.

### Create a new Evaluation
First we will import the the TEEHR Evaluation class and create a new instance that points to a directory where the evaluation data will be stored

In [2]:
from teehr import Evaluation
from pathlib import Path
import shutil

# Define the directory where the Evaluation will be created
test_eval_dir = Path(Path().home(), "temp", "03_introduction_class")
shutil.rmtree(test_eval_dir, ignore_errors=True)

# Create an Evaluation object and create the directory
ev = Evaluation(dir_path=test_eval_dir, create_dir=True)

# Enable logging
ev.enable_logging()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/11 08:52:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Clone Evaluation Data form S3
As mentioned above, for this exercise we will be cloning a complete Evaluation dataset from the TEEHR S3 bucket.  First we will list the available Evaluations and then we will clone the `p0_2_location_example` evaluation which is  a small example Evaluation that only contains 2 gages.

In [3]:
# List the evaluations in the S3 bucket
ev.list_s3_evaluations()

,name,description,url
0,p0_2_location_example,Example evaluation datsets with 2 USGS gages,s3a://ciroh-rti-public-data/teehr-data-warehou...
1,p1_camels_daily_streamflow,Daily average streamflow at ther Camels basins,s3a://ciroh-rti-public-data/teehr-data-warehou...
2,p2_camels_hourly_streamflow,Hourly instantaneous streamflow at ther Camels...,s3a://ciroh-rti-public-data/teehr-data-warehou...
3,p3_usgs_hourly_streamflow,Hourly instantaneous streamflow at USGS CONUS ...,s3a://ciroh-rti-public-data/teehr-data-warehou...


In [4]:
# Clone the p0_2_location_example evaluation from the S3 bucket
ev.clone_from_s3("p0_2_location_example")

24/11/11 08:52:35 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/11/11 08:52:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Now that we have cloned the `p0_2_location_example` evaluation, lets take a look at the data that was cloned from S3, specifically the `dataset` directory.  You can see that the three different data groups are stored in slightly different ways.  
- The domain tables (units, variables, configurations, attributes) are stored as *.csv files
- The location tables (locations, location_attributes, location_crosswalks) are stored as parquet files without hive partitioning
- The timeseries tables (primary_timeseries, secondary_timeseries, joined_timeseries) are stored as parquet files with hive partitioning

In [21]:
# Note, if you don't have tree installed and don't want to install it (i.e., you are on Windows),
# you can uncomment the next two lines and comment the third to use a Python function roughly does the same thing.

# from teehr.evaluation.utils import print_tree
# print_tree(ev.dataset_dir, exclude_patterns=[".*", "_*"])

!tree $HOME/temp/03_introduction_class/dataset -I ".*|_*"

/Users/mdenno/temp/03_introduction_class/dataset
├── attributes
│   └── part-00000-6b1674d9-f697-431f-a8b0-5fd4ad736b95-c000.csv
├── configurations
│   └── part-00000-c115ed3c-8def-45c6-8217-1d3efc9d427e-c000.csv
├── joined_timeseries
│   └── configuration_name=nwm30_retrospective
│       └── variable_name=streamflow_hourly_inst
│           └── part-00000-321fe0ea-bf64-4618-a24e-709bf8264b74.c000.snappy.parquet
├── location_attributes
│   └── part-00000-67d78cfc-82f0-433a-89b4-cdbdfdec3461-c000.snappy.parquet
├── location_crosswalks
│   └── part-00000-b128b985-bd61-47c8-8a89-b046dc4b8fc6-c000.snappy.parquet
├── locations
│   └── part-00000-1d4e966b-d413-4bb7-8b04-16c08670772b-c000.snappy.parquet
├── primary_timeseries
│   └── configuration_name=usgs_observations
│       └── variable_name=streamflow_hourly_inst
│           └── part-00000-60a5eb34-d348-4518-bea6-bff0942aa9f6.c000.snappy.parquet
├── secondary_timeseries
│   └── configuration_name=nwm30_retrospective
│       └── variable_n

### Table Classes
The TEEHR Evaluation class contains different sub-classes that are used to oragnize class methods into logical groups.  One of these types of sub-classes is the "table" sub-classes which contain methods for interacting with the data tables. Each of the tables in the Evaluation dataset has a respective sub-class with the table name.
```
ev.units
ev.attributes
ev.variables
ev.configurations
ev.locations
ev.location_attributes
ev.location_crosswalks
ev.primary_timeseries
ev.secondary_timeseries
ev.joined_timeseries
```
Each of the table sub-classes then has methods to add or insert new data as well as methods to query the table to get data out.  These are documented in the API documentation. For now, because all the tables are relatively small, we will just use the `to_pandas()` method and then the `head()` method on the Pandas DataFrame to see asn example of the data that is returned.  In an actual evaluation setting, with lots of data in the TEEHR dataset, you would likely want to include a `filter()` method to reduce the amount of data you are querying and putting into memory.

In [20]:
ev.units.to_pandas().head()

,name,long_name
0,m^3/s,Cubic Meters Per Second
1,ft^3/s,Cubic Feet Per Second
2,km^2,Square Kilometers
3,mm/s,Millimeters Per Second


In [10]:
ev.attributes.to_pandas().head()

,name,type,description
0,2_year_flow,continuous,Two year flow rate in m^3/s
1,frac_urban,categorical,frac_urban
2,high_prec_freq,categorical,high_prec_freq
3,pet_mean,categorical,pet_mean
4,ecoregion_L2,categorical,ecoregion_L2


In [11]:
ev.variables.to_pandas().head()

,name,long_name
0,streamflow_hourly_inst,Hourly Instantaneous Streamflow
1,streamflow_daily_mean,Daily Mean Streamflow
2,rainfall_hourly_rate,Hourly Rainfall Rate


In [12]:
ev.configurations.to_pandas().head()

,name,type,description
0,nwm30_retrospective,secondary,NWM 3.0 Retrospective
1,nwm22_retrospective,secondary,NWM 2.2 Retrospective
2,nwm21_retrospective,secondary,NWM 2.1 Retrospective
3,nwm20_retrospective,secondary,NWM 2.0 Retrospective
4,usgs_observations,primary,USGS Observations


In [13]:
ev.locations.to_pandas().head()

,id,name,geometry
0,usgs-14316700,"STEAMBOAT CREEK NEAR GLIDE, OR",b'\x01\x01\x00\x00\x00\x9f\xcc?\xfa\xa6\xae^\x...
1,usgs-14138800,"BLAZED ALDER CREEK NEAR RHODODENDRON, OR",b'\x01\x01\x00\x00\x00\xb7\xday\xd1\ry^\xc0\x1...


In [14]:
ev.location_attributes.to_pandas().head()

,location_id,attribute_name,value
0,usgs-14316700,aridity,0.501305088892464
1,usgs-14138800,aridity,0.270945979187767
2,usgs-14316700,dom_land_cover,Evergreen Needleleaf Forest
3,usgs-14138800,dom_land_cover,Evergreen Needleleaf Forest
4,usgs-14316700,dom_land_cover_frac,0.999999999999874


In [15]:
ev.primary_timeseries.to_pandas().head()

,reference_time,value_time,value,unit_name,location_id,configuration_name,variable_name
0,NaT,2000-09-30 20:00:00,3.341388,m^3/s,usgs-14138800,usgs_observations,streamflow_hourly_inst
1,NaT,2000-09-30 21:00:00,3.992675,m^3/s,usgs-14138800,usgs_observations,streamflow_hourly_inst
2,NaT,2000-09-30 22:00:00,4.445745,m^3/s,usgs-14138800,usgs_observations,streamflow_hourly_inst
3,NaT,2000-09-30 23:00:00,5.408518,m^3/s,usgs-14138800,usgs_observations,streamflow_hourly_inst
4,NaT,2000-10-01 00:00:00,5.606736,m^3/s,usgs-14138800,usgs_observations,streamflow_hourly_inst


In [16]:
ev.location_crosswalks.to_pandas().head()

,primary_location_id,secondary_location_id
0,usgs-14316700,nwm30-23894572
1,usgs-14138800,nwm30-23736071


In [17]:
ev.secondary_timeseries.to_pandas().head()

,reference_time,value_time,value,unit_name,location_id,configuration_name,variable_name
0,NaT,2000-09-30 20:00:00,0.38,m^3/s,nwm30-23894572,nwm30_retrospective,streamflow_hourly_inst
1,NaT,2000-09-30 20:00:00,0.06,m^3/s,nwm30-23736071,nwm30_retrospective,streamflow_hourly_inst
2,NaT,2000-09-30 21:00:00,0.38,m^3/s,nwm30-23894572,nwm30_retrospective,streamflow_hourly_inst
3,NaT,2000-09-30 21:00:00,0.06,m^3/s,nwm30-23736071,nwm30_retrospective,streamflow_hourly_inst
4,NaT,2000-09-30 22:00:00,0.38,m^3/s,nwm30-23894572,nwm30_retrospective,streamflow_hourly_inst


### Querying
Above, we just used the `to_pandas()` method on each table in the The underlying query engine for TEEHR is PySpark.  Each of the table sub-classes can return data as either a Spark DataFrame (using the `to_sdf()` method) or as a Pandas DataFrame (using the `to_pandas()` method).  The location data tables have an additional method that returns a GeoPandas DataFrame (using the `to_geopandas()` method) where the geometry bytes column has been converted to a proper WKT geometry column.

Note: PySpark itself is "lazy loaded" meaning that it does not actually run the query until the data is needed for display, plotting, etc.  Therefore, if you just use the `to_sdf()` method, you do not get the data but rather a lazy Spark DataFrame that can be used with subsequent Spark operations that will all be evaluated when the results are requested.  Here we show how to get the Spark DataFrame and show the data but there are many other ways that the lazy Spark DataFrame can be used in subsequent operations that are beyond the scope of this document.

In [ ]:
# Query the locations and return as a lazy Spark DataFrame.
ev.locations.to_sdf()

In [ ]:
# Query the locations and return as a Spark DataFrame but tell Spark to show the data.
ev.locations.to_sdf().show()

In [ ]:
# Query the locations and return as a Pandas DataFrame.
# Note that the geometry column is shown as a byte string.
ev.locations.to_pandas()

In [ ]:
# Query the locations and return as a GeoPandas DataFrame.
# Note that the geometry column is now a proper WKT geometry column.
ev.locations.to_geopandas()

### Filter and Order
As noted above, because the tables are a lazy loaded Spark DataFrames, we can filter and order the data before returning it as a Pandas or GeoPandas DataFrame. The filter methods take either a raw SQL string, a filter dictionary or a FilterObject, Operator and field enumeration. Using an FilterObject, Operator and field enumeration is probably not a common pattern for most users, but it is used internally to validate filter arguments and is available to users if they would like to use it.

In [ ]:
# Filter using a raw SQL string
ev.locations.filter("id = 'usgs-14316700'").to_geopandas()

In [ ]:
# Filter using a dictionary
ev.locations.filter({
    "column": "id",
    "operator": "=",
    "value": "usgs-14316700"
}).to_geopandas()

In [ ]:
# Import the LocationFilter and Operators classes
from teehr import LocationFilter, Operators

# Get the field enumeration
fields = ev.locations.field_enum()

# Filter using the LocationFilter class
lf = LocationFilter(
    column=fields.id,
    operator=Operators.eq,
    value="usgs-14316700"
)
ev.locations.filter(lf).to_geopandas()

This same approach can be used to query the other tables in the evaluation dataset.  There are also other methods that we did not explore and users are encourgaed to checkout the TEEHR API documentation as well as the PySpark documentation for a more indepth understanding of what happens in the background.